In [ ]:
%cd ..

In [ ]:
%cd ..

/data/apperception/apperception


In [2]:
%cd ..

/home/local/apperception/.installs/mambaforge/envs/apperception/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
YOLOv5 🚀 2022-11-23 Python-3.10.8 torch-1.13.0+cu117 CUDA:0 (NVIDIA TITAN Xp, 12196MiB)

Using cache found in /data/apperception/apperception/weights/ultralytics_yolov5_master


Using cuda:0


YOLOv5 🚀 2022-11-30 Python-3.10.8 torch-1.13.0+cu117 CUDA:0 (NVIDIA TITAN Xp, 12196MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [ ]:
import json
import os
import pickle

from optimized_ingestion.camera_config import camera_config
from optimized_ingestion.payload import Payload
from optimized_ingestion.pipeline import Pipeline
from optimized_ingestion.stages.in_view import InView
from optimized_ingestion.stages.decode_frame.parallel_decode_frame import ParallelDecodeFrame
from optimized_ingestion.stages.decode_frame.decode_frame import DecodeFrame
from optimized_ingestion.stages.detection_2d.yolo_detection import YoloDetection
from optimized_ingestion.stages.filter_car_facing_sideway import FilterCarFacingSideway
from optimized_ingestion.stages.detection_estimation import DetectionEstimation
from optimized_ingestion.stages.tracking_2d.strongsort import StrongSORT
# from optimized_ingestion.stages.tracking_2d.optimized_strongsort import OptimizedStrongSORT
from optimized_ingestion.stages.tracking_2d.tracking_2d import Tracking2D
from optimized_ingestion.stages.detection_3d.from_2d_and_road import From2DAndRoad
from optimized_ingestion.stages.tracking_3d.tracking_3d import Tracking3DResult
# from optimized_ingestion.trackers.yolov5_strongsort_osnet_tracker import TrackingResult
from optimized_ingestion.video import Video

In [3]:
from optimized_ingestion.cache import disable_cache
disable_cache()

In [4]:
BOSTON_VIDEOS = [
#     "scene-0757-CAM_FRONT",
    # "scene-0103-CAM_FRONT",
#     "scene-0553-CAM_FRONT",
    # "scene-0665-CAM_FRONT",
#     "scene-0655-CAM_FRONT",
#     "scene-0655-CAM_FRONT_RIGHT",
#     "scene-0655-CAM_BACK_RIGHT",
#     "scene-0553-CAM_FRONT_LEFT"
#     "scene-0103-CAM_FRONT"
]

NUSCENES_PROCESSED_DATA = "NUSCENES_PROCESSED_DATA"

In [5]:
pipeline = Pipeline()
pipeline.add_filter(filter=InView(distance=10, segment_type="intersection"))
# pipeline.add_filter(filter=Stopped(min_stopped_frames=2, stopped_threshold=1.0))
pipeline.add_filter(filter=ParallelDecodeFrame())
pipeline.add_filter(filter=YoloDetection())

pipeline.add_filter(filter=From2DAndRoad())
pipeline.add_filter(filter=DetectionEstimation(lambda x: x.road_type == 'intersection'))  # 5 Frame p Second
pipeline.add_filter(filter=StrongSORT())  # 2 Frame p Second

# pipeline.add_filter(filter=From2DAndRoad())
# pipeline.add_filter(filter=FilterCarFacingSideway())

if NUSCENES_PROCESSED_DATA in os.environ:
    DATA_DIR = os.environ[NUSCENES_PROCESSED_DATA]
else:
    DATA_DIR = "/work/apperception/data/nuScenes/full-dataset-v1.0/Mini"
with open(os.path.join(DATA_DIR, "videos/boston-seaport", "frames.pickle"), "rb") as f:
    videos = pickle.load(f)

num_video = 0
for name, video in videos.items():
#     if name not in BOSTON_VIDEOS:
#         continue
#     if not name.endswith('CAM_FRONT'):
#         continue
    if 'CAM_FRONT' not in name:
        continue

    print(name, '--------------------------------------------------------------------------------')
    frames = Video(
        os.path.join(DATA_DIR, "videos/boston-seaport", video["filename"]),
        [camera_config(*f, 0) for f in video["frames"]],
        video["start"],
    )

    output = pipeline.run(Payload(frames))
    num_video += 1

total_runtime = 0
stage_runtimes = []
benchmarks = []
for stage in pipeline.stages:
    stage_runtimes.append({
        "stage": stage.classname(),
        "runtimes": stage.runtimes,
    })
    total_runtime += sum([run['runtime'] for run in stage.runtimes])

benchmarks.append({
    'stage_runtimes': stage_runtimes,
    'total_runtime': total_runtime
})
if num_video:
    benchmarks.append({'average runtime': sum([b['total_runtime'] for b in benchmarks])/num_video})


YOLOv5 🚀 2022-11-23 Python-3.10.8 torch-1.13.0+cu117 CUDA:0 (NVIDIA TITAN Xp, 12196MiB)

Using cache found in /data/apperception/weights/ultralytics_yolov5_master
YOLOv5 🚀 2022-11-30 Python-3.10.8 torch-1.13.0+cu117 CUDA:0 (NVIDIA TITAN Xp, 12196MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


scene-0103-CAM_FRONT_LEFT --------------------------------------------------------------------------------
Stage:  DecodeFrame.ParallelDecodeFrame


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 80/80 [00:15<00:00,  5.09it/s]


None
388
  filtered frames: 100.0%
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
Stage:  Detection2D.YoloDetection


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 388/388 [00:04<00:00, 89.69it/s]


...

None
397
  filtered frames: 100.0%
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
Stage:  Detection3D.From2DAndRoad


397it [00:00, 3269.09it/s]
YOLOv5 🚀 2022-11-23 Python-3.10.8 torch-1.13.0+cu117 CUDA:0 (NVIDIA TITAN Xp, 12196MiB)



None
397
  filtered frames: 100.0%
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
Stage:  Tracking2D.StrongSORT
Successfully loaded pretrained weights from "/data/apperception/apperception/weights/osnet_x0_25_msmt17.pt"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 397/397 [00:19<00:00, 20.47it/s]


None
397
  filtered frames: 100.0%
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK


In [ ]:
with open("./outputs/fig13_full_optimization.txt", "w") as f3:
    json.dump(benchmarks, f3)

In [ ]:
pipeline = Pipeline()
# pipeline.add_filter(filter=InView(distance=10, segment_type="intersection"))
# pipeline.add_filter(filter=Stopped(min_stopped_frames=2, stopped_threshold=1.0))
pipeline.add_filter(filter=ParallelDecodeFrame())
pipeline.add_filter(filter=YoloDetection())

pipeline.add_filter(filter=From2DAndRoad())
# pipeline.add_filter(filter=DetectionEstimation(lambda x: x.road_type == 'intersection'))  # 5 Frame p Second
pipeline.add_filter(filter=StrongSORT())  # 2 Frame p Second

# pipeline.add_filter(filter=From2DAndRoad())
# pipeline.add_filter(filter=FilterCarFacingSideway())

if NUSCENES_PROCESSED_DATA in os.environ:
    DATA_DIR = os.environ[NUSCENES_PROCESSED_DATA]
else:
    DATA_DIR = "/work/apperception/data/nuScenes/full-dataset-v1.0/Mini"
with open(os.path.join(DATA_DIR, "videos/boston-seaport", "frames.pickle"), "rb") as f:
    videos = pickle.load(f)

num_video = 0
for name, video in videos.items():
#     if name not in BOSTON_VIDEOS:
#         continue
#     if not name.endswith('CAM_FRONT'):
#         continue
    if 'CAM_FRONT' not in name:
        continue

    print(name, '--------------------------------------------------------------------------------')
    frames = Video(
        os.path.join(DATA_DIR, "videos/boston-seaport", video["filename"]),
        [camera_config(*f, 0) for f in video["frames"]],
        video["start"],
    )

    output = pipeline.run(Payload(frames))
    num_video += 1

total_runtime = 0
stage_runtimes = []
benchmarks = []
for stage in pipeline.stages:
    stage_runtimes.append({
        "stage": stage.classname(),
        "runtimes": stage.runtimes,
    })
    total_runtime += sum([run['runtime'] for run in stage.runtimes])

benchmarks.append({
    'stage_runtimes': stage_runtimes,
    'total_runtime': total_runtime
})
if num_video:
    benchmarks.append({'average runtime': sum([b['total_runtime'] for b in benchmarks])/num_video})

In [ ]:
with open("./outputs/fig13_without_optimization.txt", "w") as f3:
    json.dump(benchmarks, f3)

In [ ]:
with open("./outputs/benchmark-with-estimation.json", "w") as f3:
    json.dump(with_estimation_benchmarks, f3)